# About this Page
## What is new in this application?(newest change at top of list)
- Exposure quality flag added to Exposure Detail (accessed by following row links in Data Log)
- Bullet proof parameters
- Do not show Time Loss info unless it is greater than zero (Narrative log)
- Do not show software or hardware components (they are rarely used anyhow)
- <font style="background-color:#ffff99">Embedded comments and questions to reader (Yellow background)</font>
- This page tries to work around existing problems with APIs used for source data. (There is a ticket to fix the APIs)
- <font style="background-color:red; color:#90EE90">Errors from APIs are displayed in this page (Red background).</font>


In [ ]:
# Parameters.
# Times Square replaces this cell with the user's parameters.
# So, don't put anything else here!

# day_obs values: TODAY, YESTERDAY, YYYY-MM-DD
# Report on observing nights that start upto but not included this day.
#!day_obs = '2024-09-25' # Value to use for local testing (Summit)
day_obs = "YESTERDAY"  # TODO Change to 'YESTERDAY' and 'TODAY' to test with default before push

# Total number of days of data to display (ending on day_obs)
number_of_days = "1"  # TODO Change to '1' to test with default before push

verbose = "false"  # TODO change to false before push, else true

In [ ]:
# IMPORT everything
import datetime as dt
import os
from collections import defaultdict
from pprint import pformat, pp
from urllib.parse import urlencode, urlparse
import pandas as pd
from IPython.display import HTML

# When running under Times Square, install pkg from github.
# Otherwise use what is installed locally (intended to be dev editiable pkg)
if os.environ.get("EXTERNAL_INSTANCE_URL"):
    dev_msg = (
        'Installing "lsst.ts.logging_and_reporting" from github using "prototype" branch. \n'
        'TODO: Make the need for this go away by getting Logging_and_Reporting installed in RSP.'
    )
    !pip install --upgrade git+https://github.com/lsst-ts/ts_logging_and_reporting.git@prototype > /dev/null 2>&1
else:
    dev_msg = 'Imported lsst.ts.logging_and_reporting from local packages.'
from lsst.ts.logging_and_reporting.all_sources import AllSources
from lsst.ts.logging_and_reporting.all_reports import AllReports
import lsst.ts.logging_and_reporting.utils as ut
from lsst.ts.logging_and_reporting.reports import md, mdlist, mdpathlink
from lsst.ts.logging_and_reporting.reports import html_draft, html_beta

In [ ]:
# Set default env to "usdf" and try before PUSH to repo.
# The default provided here is for local testing. Under Times Square it is ignored.
server = os.environ.get(
    "EXTERNAL_INSTANCE_URL", 
    ut.Server.usdf) # TODO try with "usdf" before push (also "summit")

In [ ]:
# Validate parameters, return usable ones
usable, error = ut.fallback_parameters(day_obs, number_of_days, verbose)
if error:
    print(error)

date = ut.get_datetime_from_dayobs_str(usable['day_obs'])
# date:  is EXCLUSIVE (upto, but not including)
days = usable['number_of_days']
verbose = usable['verbose']

# Thus: [min_day_obs,max_day_obs)
# Format: string, YYYY-MM-DD
min_date = date - dt.timedelta(days=days - 1)
max_date = date + dt.timedelta(days=1)
min_day_obs = min_date.strftime("%Y-%m-%d")  # Inclusive
max_day_obs = max_date.strftime("%Y-%m-%d")  # prep for Exclusive

In [ ]:
# Read records from (almost) all sources
timer = ut.Timer()
allsrc = AllSources(
    server_url=server,
    min_dayobs=min_day_obs,
    max_dayobs=max_day_obs,
    verbose=verbose,                
    limit=5000,
    exclude_instruments = []      # TODO change to empty list before push
)
allrep = AllReports(allsrc=allsrc)

----------

In [ ]:
instrum_str = ", ".join(list(allsrc.exp_src.instruments.keys()))
md(f"# Showing data for {min_date.date()} to {max_date.date()} for {instrum_str}")
md(f"Most recently run on: **{dt.datetime.now().replace(microsecond=0).isoformat(sep=' ')}**")

# Table of Contents
* [Night Report](#Night-Report-BETA)
    - AuxTel
    - Simonyi
* [Almanac](#Almanac-BETA)
* [Summary plots of whole night](#Summary-plots-of-whole-night-DRAFT)
* [Time Accounting](#Time-Accounting-DRAFT)
* [Jira Tickets](#Jira-Tickets-BETA)
    - AuxTel
    - Simonyi
* [BLOCKS Observed](#BLOCKS-Observed-DRAFT)
    - AuxTel
    - Simonyi
* [Data Log](#Data-Log-BETA)
* [Narrative Log](#Narrative-Log-BETA)
* [Developer Only Section](#Developer-Only-Section-REMOVE)

## Night Report <font style="background-color:green; color:white; font-size:20px">BETA</font> 
#### <font style="background-color:#ffff99">Possible Changes</font>
- Word wrap instead of chopping words at end of line?
- (Add summary from AI)

In [ ]:
# Night Report
# Display time log
allrep.nig_rep.time_log_as_markdown()

## Almanac <font style="background-color:green; color:white; font-size:20px">BETA</font> 
<font style="background-color:#ffff99">
Add column for Chile timezone?
</font>

In [ ]:
# Display various almanac values (for moon, sun)
#!rep.AlmanacReport().day_obs_report(min_day_obs)
allrep.alm_rep.day_obs_report()

## Summary plots of whole night <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>

(content not yet defined in storyboard)
#### <font style="background-color:#ffff99">Possible Features</font>
- plot_observation_gap_detail

In [ ]:
# Plot Observation (Exposure) gaps
# rollup,detail = allsrc.get_observation_gaps()
# allrep.plot_observation_gap_detail(detail)

## DDV <font style="background-color:green; color:white; font-size:20px">BETA</font>

In [ ]:
DDV = (
    f"{server}/rubintv-dev/ddv/index.html"
    if "summit" in server
    else f"{server}/rubintv/ddv/index.html"
)
md(f"Access DDV part of RubinTV: {DDV}")

## Time Accounting <font style="background-color:green; color:white; font-size:20px">BETA</font>
TODO:
- Time-loss for Fault and Weather  (very rarely in narrativelog/messages)
- SlewTime from TMAEvent (or similar)

SlewTime (and probably others) from EFD are only EXPECTED (topic="lsst.sal.Scheduler.logevent_target"), not ACTUAL.
To get ACTUAL time, we need to use something like 
[**TMAEvent**](https://github.com/lsst-sitcom/summit_utils/blob/f901ca4943c082500a1f357eb070a4971925c476/python/lsst/summit/utils/tmaUtils.py#L1195) 
to combine multiple real events into a synthetic event.  See DM-46102.


In [ ]:
# Exposure Tally
tally = await allsrc.night_tally_observation_gaps()
if tally:
    display(pd.DataFrame(tally))

- (1) *Expected* slew time as per Schedular
- (2) There is no practical way to get detector read-out time.  A value of 2.3 seconds was inferred from experiments.

## Jira Tickets <font style="background-color:green; color:white; font-size:20px">BETA</font> 
#### <font style="background-color:#ffff99">Possible Changes</font>
- Observing Operations - OBS
- TODO: better tickets using Jira API to get selected fields (which) from tickets

In [ ]:
if allsrc.urls:
    for url in allsrc.urls:
        md(f"- {mdpathlink(url)}")

In [ ]:
# Display Jira BLOCKS
front = "https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/"
tickets = allsrc.nig_src.nightly_tickets()
if tickets:
    mdstr = "#### Nightly Jira BLOCKs"
    for day, url_list in tickets.items():
        mdstr += f"\n- {day}"
        for ticket_url in url_list:
            mdstr += f'\n    - [{ticket_url.replace(front,"")}]({ticket_url})'
    md(mdstr)
else:
    endpoint_url = allsrc.nig_src.status["reports"]["endpoint_url"]
    md(f"No tickets found using: [API Data]({endpoint_url}) in `confluence_url`.")

## BLOCKS Observed <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>
- TODO: BLOCKS observed

## Data Log <font style="background-color:green; color:white; font-size:20px">BETA</font>
Count the number of exposures (tally) for each instrument. For instruments that have exposures, partition the tally by various attributes and the exposure quality flags.

In [ ]:
# Exposure Report
for instrum in allsrc.exp_src.exposures.keys():
    md(f'#### {instrum}: {len(allsrc.exp_src.exposures[instrum])} total exposures')
    df_dict = allsrc.fields_count_exposure(instrum)
    for field_name,df in df_dict.items():
        if df.empty:
            continue
        md('##### ' + field_name.title().replace('_',' '))
        display(HTML(df.style.hide().to_html(escape=False)))

## Narrative Log <font style="background-color:green; color:white; font-size:20px">BETA</font> 
#### <font style="background-color:#ffff99">Possible Changes</font>
- Group messages by hour? Might be great when there many simple messages minutes apart. Less so when tracebacks are included.
- Truncate text messages to N chars (ending with "..." as link if truncated, link goes to page with full message record including text and all attributes)
- Detect some kinds of "blocks" and render in smaller text (+ other format changes)

In [ ]:
# Narrative Report
# Time Log
allrep.nar_rep.time_log_as_markdown()

-----------
------------

# Developer Only Section <font style="background-color:red; olor:black; font-size:20px">REMOVE</font>
Contains stuff only expected to be useful to developers.

This may also contain sections that have moved out of the user section because they are no longer defined in the Storyboard.

## Overview 


In [ ]:
# Display overview of Report context
try:
    import lsst.ts.logging_and_reporting.version

    lrversion = lsst.ts.logging_and_reporting.version.__version__
except:
    lrversion = "LIVE"

try:
    from lsst.summit.utils import ConsDbClient

    have_consdb = True
except:
    have_consdb = False
md(
    f"""## Project-Wide Night(s) Report 
- Run on logs and databases from **{server}/**
- Report **{days} observing night(s)** with the last reported night starting on **{date}**.
- min_dayobs={allsrc.min_dayobs!r}, max_dayobs={allsrc.max_dayobs!r}
- Using ***Prototype* Logging and Reporting** Version: **{lrversion}**
- {have_consdb = }
- {dev_msg}
- {allsrc.nig_src.timeout=}
""")

## Data Status
This may be useful for diagnosing reports that give unexpected results.

In [ ]:
df = pd.DataFrame(allsrc.get_data_status())
table = df.to_html(
    index=False, 
    render_links=True,
    escape=False)
display(HTML(table))

## This report uses the following data sources
- NightReport
- Exposurelog
- Narrativelog
- EFD
- ConsDB
- (DDV)
- (Almanac from Astroplan)

## Where was this run?

The environments that have everything needed to run this page are:

    https://summit-lsp.lsst.codes
    https://usdf-rsp-dev.slac.stanford.edu

However, Times Square does not run on the Summit. It does run on USDF-dev.


## Section overviews moved here

In [ ]:
# Night Report Overview
allrep.nig_rep.overview()
# Exposure Report Overview
allrep.exp_rep.overview()
# Narrative Report Overview
allrep.nar_rep.overview()

In [ ]:
# Conditionally display our current ability to connect to all needed endpoints.
if False and not os.environ.get("EXTERNAL_INSTANCE_URL"):
    md("## Dashboard")
    md("(This is not done when running under Times Square.)")
    %run ./dashboard.ipynb

In [ ]:
# Time Log
#! allrep.exp_rep.time_log_as_markdown()

## Finale

In [ ]:
print(f'Elapsed time (excluding code import): {timer.toc:.1f} seconds')
print(f"Finished {str(dt.datetime.now().replace(microsecond=0))} UTC")